In [270]:
import pandas as pd
import numpy as np
import requests
import re
from bs4 import BeautifulSoup


# Cazoo

## Testing

In [3]:
url = 'https://www.cazoo.co.uk/cars/'
res = requests.get(url)
res

<Response [200]>

In [4]:
soup = BeautifulSoup(res.content, 'html.parser')
table = soup.find(attrs={'class':"resultsstyles__ResultsList-sc-11kjmxu-0 ezjHRs"})
tbody = table.find_all('li')
tr = tbody[2]
make_model = tr.find(attrs={'class':'vehicle-cardstyles__CustomTitle-sc-1bxv5iu-8 vEuQf'}).text
version = tr.find(attrs={'class':'vehicle-cardstyles__DisplayVariant-sc-1bxv5iu-9 fxjZPj'}).text
dets= tr.find_all(attrs={'data-test-id':'tags'})
version = version.split()
engine = ''.join(version[0])
trim = " ".join(version[1:])
price = tr.find(attrs={'data-test-id':'card-pricing-full-price-gb'}).text

In [5]:
split = make_model.split()
split[0]

'Skoda'

In [71]:
url = 'https://www.cazoo.co.uk/cars/audi/'
res = requests.get(url)
soup = BeautifulSoup(res.content, 'html.parser')
pageselection = soup.find(attrs={'aria-label':'Pagination'})
numpages = pageselection.find_all(attrs={'class':'cui__sc-j9mjm2-0 cui__sc-j9mjm2-3 jkCaUj dUXlEO'})
numpages

[<li class="cui__sc-j9mjm2-0 cui__sc-j9mjm2-3 jkCaUj dUXlEO"><a data-test-id="pagination-link-2" href="/cars/audi/?page=2"><span class="visuallyhidden">Go to page<!-- --> </span>2</a></li>,
 <li class="cui__sc-j9mjm2-0 cui__sc-j9mjm2-3 jkCaUj dUXlEO"><a data-test-id="pagination-link-3" href="/cars/audi/?page=3"><span class="visuallyhidden">Go to page<!-- --> </span>3</a></li>,
 <li class="cui__sc-j9mjm2-0 cui__sc-j9mjm2-3 jkCaUj dUXlEO"><a data-test-id="pagination-link-4" href="/cars/audi/?page=4"><span class="visuallyhidden">Go to page<!-- --> </span>4</a></li>,
 <li class="cui__sc-j9mjm2-0 cui__sc-j9mjm2-3 jkCaUj dUXlEO"><a data-test-id="pagination-link-5" href="/cars/audi/?page=5"><span class="visuallyhidden">Go to page<!-- --> </span>5</a></li>]

## Automating

In [6]:
manufactures = {'abarth', 'alfa-romeo', 'audi', 'bmw', 'citroen', 'cupra', 'dacia', 'ds', 'fiat',
               'ford', 'honda', 'hyundai','jaguar', 'jeep', 'kia', 'land-rover',
               'lexus', 'mazda', 'mg','mini','mitsubishi','nissan','peugeot',
               'polestar','renault','seat','skoda','smart','subaru','suzuki','tesla',
               'toyota','vauxhall','volkswagen','volvo'}

In [93]:
vehicles = []
for make in manufactures:
    url = 'https://www.cazoo.co.uk/cars/' + make
    print(make)
    res = requests.get(url)
    soup = BeautifulSoup(res.content, 'html.parser')
    table = soup.find(attrs={'data-test-id':"results-list"})
    tbody = table.find_all('li')
    tbody = tbody[2:]
    pageselect = soup.find(attrs={'aria-label':'Pagination'})
    numpages = pageselection.find_all(attrs={'class':'cui__sc-j9mjm2-0 cui__sc-j9mjm2-3 jkCaUj dUXlEO'})
    n = len(numpages) + 1
    for tr in tbody:
        make_model = tr.find(attrs={'data-test-id':'card-title'}).text
        makemod = make_model.split()
        make = makemod[0]
        model = ' '.join(makemod[1:])
        version = tr.find(attrs={'class':'vehicle-cardstyles__DisplayVariant-sc-1bxv5iu-9 fxjZPj'}).text
        version = version.split()
        engine = ''.join(version[0])
        trim = " ".join(version[1:])
        dets = tr.find_all(attrs={'data-test-id':'tags'})
        details = dets[0]
        details_str = str(details)  # Convert the first object in the list to a string
        matches = re.findall(r'(\d[\d,]* miles|\d{4} reg|Auto|Manual|Petrol|Diesel|Hybrid|\
        Electric)', details_str)
        trans = matches[2] if 'Auto' in matches[2] or 'Manual' in matches[2] else None
        mileage = matches[0] if 'miles' in matches[0] else None
        registration = matches[1] if 'reg' in matches[1] else None
       
        price = tr.find(attrs={'data-test-id':'card-pricing-full-price-gb'}).text
        vehicle = {'make':make, 'model': model, 'engine_size':engine,'trim':trim, 'mileage':mileage,
               'reg_year':registration, 'transmission_type':trans, 'fuel_type':fuel_type,
               'price':price
              }
        vehicles.append(vehicle)
    for i in range(2, n):
        url = url + '/?page=' + str(i)
        print(make, i)
        res = requests.get(url)
        soup = BeautifulSoup(res.content, 'html.parser')
        table = soup.find(attrs={'data-test-id':"results-list"})
        
        if table != None:
            tbody = table.find_all('li')
            tbody = tbody[2:]
            count += 1
            print(count)
            for tr in tbody:
                make_model = tr.find(attrs={'data-test-id':'card-title'}).text
                makemod = make_model.split()
                make = makemod[0]
                model = ' '.join(makemod[1:])
                version = tr.find(attrs={'class':'vehicle-cardstyles__DisplayVariant-sc-1bxv5iu-9 fxjZPj'}).text
                version = version.split()
                engine = ''.join(version[0])
                trim = " ".join(version[1:])
                dets = tr.find_all(attrs={'data-test-id':'tags'})
                details = dets[0]
                details_str = str(details)  # Convert the first object in the list to a string
                matches = re.findall(r'(\d[\d,]* miles|\d{4} reg|Auto|Manual|Petrol|Diesel|Hybrid|\
                Electric)', details_str)
                trans = matches[2] if 'Auto' in matches[2] or 'Manual' in matches[2] else None
                mileage = matches[0] if 'miles' in matches[0] else None
                registration = matches[1] if 'reg' in matches[1] else None
                
                price = tr.find(attrs={'data-test-id':'card-pricing-full-price-gb'}).text
                vehicle = {'make':make, 'model': model, 'engine_size':engine,'trim':trim, 'mileage':mileage,
               'reg_year':registration, 'transmission_type':trans, 'fuel_type':fuel_type,
               'price':price
              }
                vehicles.append(vehicle)
                counts = {'make':make, 'Count':count}

volvo
Volvo 2
115
Volvo 3
116
Volvo 4
117
mitsubishi
Mitsubishi 2
Mitsubishi 3
Mitsubishi 4
volkswagen
Volkswagen 2
118
Volkswagen 3
119
Volkswagen 4
120
audi
Audi 2
121
Audi 3
122
Audi 4
123
renault
Renault 2
124
Renault 3
125
Renault 4
126
mazda
Mazda 2
Mazda 3
Mazda 4
vauxhall
Vauxhall 2
127
Vauxhall 3
128
Vauxhall 4
129
abarth
Abarth 2
Abarth 3
Abarth 4
fiat
Fiat 2
130
Fiat 3
131
Fiat 4
132
citroen
Citroen 2
133
Citroen 3
134
Citroen 4
135
jeep
Jeep 2
Jeep 3
Jeep 4
mg
MG 2
MG 3
MG 4
peugeot
Peugeot 2
136
Peugeot 3
137
Peugeot 4
138
subaru
Subaru 2
Subaru 3
Subaru 4
kia
Kia 2
139
Kia 3
140
Kia 4
141
cupra
Cupra 2
Cupra 3
Cupra 4
nissan
Nissan 2
142
Nissan 3
143
Nissan 4
144
bmw
BMW 2
145
BMW 3
146
BMW 4
147
skoda
Skoda 2
148
Skoda 3
149
Skoda 4
150
ds
DS 2
DS 3
DS 4
toyota
Toyota 2
151
Toyota 3
152
Toyota 4
153
suzuki
Suzuki 2
Suzuki 3
Suzuki 4
smart
Smart 2
Smart 3
Smart 4
land-rover
Land 2
154
Land 3
155
Land 4
156
alfa-romeo
Alfa 2
Alfa 3
Alfa 4
ford
Ford 2
157
Ford 3
158
Ford 4


In [94]:
p1s = pd.DataFrame(vehicles)
p1s


,make,model,engine_size,trim,mileage,reg_year,transmission_type,fuel_type,price
0,Volvo,V40,2.0,T2 Momentum,"42,006 miles",2019 reg,Manual,Petrol,"£12,150"
1,Volvo,V40,2.0,T2 R-Design Edition,"34,495 miles",2019 reg,Manual,Petrol,"was £13,700£13,450"
2,Volvo,V40,2.0,D2 R-Design Edition Auto,"76,673 miles",2019 reg,Auto,Petrol,"£12,900"
3,Volvo,V40,1.5,T3 Momentum Nav Plus Auto,"67,129 miles",2017 reg,Auto,Petrol,"£11,650"
4,Volvo,V40,2.0,T2 R-Design Nav,"36,756 miles",2016 reg,Manual,Petrol,"£12,250"
...,...,...,...,...,...,...,...,...,...
3671,Polestar,Polestar 2,Dual,Motor 78kWh Long Range Plus Pilot Auto 4WDE,"17,011 miles",2020 reg,Auto,Petrol,"£30,350"
3672,Polestar,Polestar 2,Dual,Motor 78kWh Long Range Plus Pilot Auto 4WDE,"17,333 miles",2021 reg,Auto,Petrol,"£31,600"
3673,Polestar,Polestar 2,Dual,Motor 78kWh Long Range Plus Pilot Auto 4WDE,"8,066 miles",2021 reg,Auto,Petrol,"£32,850"
3674,Polestar,Polestar 2,Dual,Motor 78kWh Long Range Auto 4WDE,"14,253 miles",2021 reg,Auto,Petrol,"£32,800"


In [95]:
p1s['trim'].nunique()

874

In [96]:
p1s['make'].value_counts()

BMW           188
Nissan        188
Volkswagen    188
Audi          188
Renault       188
Seat          188
Vauxhall      188
Mini          188
Hyundai       188
Jaguar        188
Ford          188
Land          188
Peugeot       188
Toyota        188
Kia           188
Skoda         188
Volvo         170
Citroen       110
Fiat           65
Lexus          37
Mitsubishi     35
Suzuki         33
MG             32
Honda          32
Mazda          30
Jeep           28
Tesla          28
Dacia          18
DS             12
Abarth          9
Smart           8
Cupra           7
Polestar        7
Alfa            6
Subaru          1
Name: make, dtype: int64

## Mercs

In [97]:
url = 'https://www.cazoo.co.uk/cars/mercedes-benz/'
res = requests.get(url)
soup = BeautifulSoup(res.content, 'html.parser')
table = soup.find(attrs={'class':"resultsstyles__ResultsList-sc-11kjmxu-0 ezjHRs"})
tbody = table.find_all('li')
tbody = tbody[2:]

In [98]:
for tr in tbody:
    make_model = tr.find(attrs={'data-test-id':'card-title'}).text
    makemod = make_model.split()
    make = makemod[0]
    model = ' '.join(makemod[1:])
    version = tr.find(attrs={'class':'vehicle-cardstyles__DisplayVariant-sc-1bxv5iu-9 fxjZPj'}).text
    version = version.split()
    engine = ''.join(version[0])
    trim = " ".join(version[1:])
    dets = tr.find_all(attrs={'data-test-id':'tags'})
    details = dets[0]
    details_str = str(details)  # Convert the first object in the list to a string
    matches = re.findall(r'(\d[\d,]* miles|\d{4} reg|Auto|Manual|Petrol|Diesel|Hybrid|\
    Electric)', details_str)
    trans = matches[2] if 'Auto' in matches[2] or 'Manual' in matches[2] else None
    mileage = matches[0] if 'miles' in matches[0] else None
    registration = matches[1] if 'reg' in matches[1] else None
    price = tr.find(attrs={'data-test-id':'card-pricing-full-price-gb'}).text
    vehicle = {'make':make, 'model': model, 'engine_size':engine,'trim':trim, 'mileage':mileage,
               'reg_year':registration, 'transmission_type':trans, 'fuel_type':fuel_type,
               'price':price
              }
    vehicles.append(vehicle)

In [101]:
for i in range(2, 7):
    url = 'https://www.cazoo.co.uk/cars/mercedes-benz/?page=' + str(i)
    res = requests.get(url)
    soup = BeautifulSoup(res.content, 'html.parser')
    table = soup.find(attrs={'class':"resultsstyles__ResultsList-sc-11kjmxu-0 ezjHRs"})
    tbody = table.find_all('li')
    tbody = tbody[2:]
    for tr in tbody:
        make_model = tr.find(attrs={'data-test-id':'card-title'}).text
        makemod = make_model.split()
        make = makemod[0]
        model = ' '.join(makemod[1:])
        version = tr.find(attrs={'class':'vehicle-cardstyles__DisplayVariant-sc-1bxv5iu-9 fxjZPj'}).text
        version = version.split()
        engine = ''.join(version[0])
        trim = " ".join(version[1:])
        dets = tr.find_all(attrs={'data-test-id':'tags'})
        details = dets[0]
        details_str = str(details)  # Convert the first object in the list to a string
        matches = re.findall(r'(\d[\d,]* miles|\d{4} reg|Auto|Manual|Petrol|Diesel|Hybrid|\
        Electric)', details_str)
        trans = matches[2] if 'Auto' in matches[2] or 'Manual' in matches[2] else None
        mileage = matches[0] if 'miles' in matches[0] else None
        registration = matches[1] if 'reg' in matches[1] else None
       
        price = tr.find(attrs={'data-test-id':'card-pricing-full-price-gb'}).text
        vehicle = {'make':make, 'model': model, 'engine_size':engine,'trim':trim, 'mileage':mileage,
               'reg_year':registration, 'transmission_type':trans, 'fuel_type':fuel_type,
               'price':price
              }
        vehicles.append(vehicle)

In [102]:
p1s = pd.DataFrame(vehicles)
p1s

,make,model,engine_size,trim,mileage,reg_year,transmission_type,fuel_type,price
0,Volvo,V40,2.0,T2 Momentum,"42,006 miles",2019 reg,Manual,Petrol,"£12,150"
1,Volvo,V40,2.0,T2 R-Design Edition,"34,495 miles",2019 reg,Manual,Petrol,"was £13,700£13,450"
2,Volvo,V40,2.0,D2 R-Design Edition Auto,"76,673 miles",2019 reg,Auto,Petrol,"£12,900"
3,Volvo,V40,1.5,T3 Momentum Nav Plus Auto,"67,129 miles",2017 reg,Auto,Petrol,"£11,650"
4,Volvo,V40,2.0,T2 R-Design Nav,"36,756 miles",2016 reg,Manual,Petrol,"£12,250"
...,...,...,...,...,...,...,...,...,...
3952,Mercedes-Benz,S Class,3.0,S350Ld V6 AMG Line 9G-Tronic+,"47,784 miles",2016 reg,Auto,Petrol,"£24,200"
3953,Mercedes-Benz,GLE Class,3.0,GLE500e V6 8.8kWh AMG Line (Premium) G-Tronic+...,"66,405 miles",2016 reg,Auto,Petrol,"£25,000"
3954,Mercedes-Benz,EQC,EQC,400 80kWh AMG Line (Premium Plus) Auto 4MATIC,"35,139 miles",2020 reg,Auto,Petrol,"£34,350"
3955,Mercedes-Benz,GLC Class,2.1,GLC250d AMG Line (Premium Plus) G-Tronic 4MATIC,"21,305 miles",2017 reg,Auto,Petrol,"£26,450"


In [103]:
p1s['make'].value_counts()

Mercedes-Benz    281
Peugeot          188
Seat             188
Mini             188
Hyundai          188
Jaguar           188
Ford             188
Land             188
Toyota           188
BMW              188
Nissan           188
Kia              188
Skoda            188
Volkswagen       188
Vauxhall         188
Audi             188
Renault          188
Volvo            170
Citroen          110
Fiat              65
Lexus             37
Mitsubishi        35
Suzuki            33
MG                32
Honda             32
Mazda             30
Jeep              28
Tesla             28
Dacia             18
DS                12
Abarth             9
Smart              8
Polestar           7
Cupra              7
Alfa               6
Subaru             1
Name: make, dtype: int64

In [104]:
p1s.to_csv('../Data/cazoo_vehicles.csv', index = False)

# BuyaCar

## Testing

In [107]:
url2 = 'https://www.buyacar.co.uk/cars'
res2 = requests.get(url2)
res2

<Response [200]>

In [229]:
soup = BeautifulSoup(res2.content)
page=soup.find(attrs={'id':'page'})
text = page.find('script').text
mileage = re.findall(r'\bmileage\b\D*(\d+)', text)
price = re.findall(r'\bprice\b\D*([\d,]+)', text)
age = re.findall(r'\bcarDateOfReg\b\D*(\d+)', text)
cars = re.findall(r'"vehicleFullName_S":\s*"([^"]*)"', text)
gearbox = re.findall(r'"gearbox":\s*"([^"]*)"', text)
fuel = re.findall(r'"dennis_fuel_type":\s*"([^"]*)"', text)
# cars[0][0:10]
test3 = []
for i in range(0, len(cars)):
    vehicle = cars[i].split()
    make = vehicle[0]
    model = vehicle[1]
    size = vehicle[2]
    trim = ' '.join(vehicle[3:])
    test1 = {
        'Make': make,
        'Model': model,
        'engine_size': size,
        'trim': trim
    }
    test3.append(test1)
test = {
    'Mileage': mileage[1:21],
    'Age': age,
    'Gearbox': gearbox,
    'Fuel Type': fuel,
    'Price': price[1:21]
}
len(mileage), len(price), len(age), len(cars), len(gearbox), len(fuel)
#test
#test3.append(test)
test3 = pd.DataFrame(test3)
test = pd.DataFrame(test)
page1 = pd.concat([test3, test], axis=1)

## Automating

In [271]:
all_cars = []
all_info = []
miles = []
prices = []
ages = []
gears = []
fuel_types=[]
for i in range(1, 1206):
    url = 'https://www.buyacar.co.uk/cars?f[0]=vehicle_type:car&offset=' + str(i) + '&limit=20'
    res = requests.get(url)
    soup = BeautifulSoup(res.content, 'html.parser')  
    page = soup.find(attrs={'id': 'page'})
    text = page.find('script').text
    
    mileage = re.findall(r'\bmileage\b\D*(\d+)', text)
    price = re.findall(r'\bprice\b\D*([\d,]+)', text)
    age = re.findall(r'\bcarDateOfReg\b\D*(\d+)', text)
    cars = re.findall(r'"vehicleFullName_S":\s*"([^"]*)"', text)
    gearbox = re.findall(r'"gearbox":\s*"([^"]*)"', text)
    fuel = re.findall(r'"dennis_fuel_type":\s*"([^"]*)"', text)
    
    cars_split = []
    for j in range(0, len(cars)):  
        vehicle = cars[j].split()
        make = vehicle[0]
        model = vehicle[1]
        size = vehicle[2]
        trim = ' '.join(vehicle[3:])
        split = {
            'Make': make,
            'Model': model,
            'engine_size': size,
            'trim': trim
        }
        cars_split.append(split)
        
    info = {
        'Age': age,
        'Gearbox': gearbox,
        'Fuel Type': fuel
    }
    miles.extend(mileage[1:21])
    prices.extend(price[2:22])
    all_cars.extend(cars_split)
    ages.extend(age)
    gears.extend(gearbox)
    fuel_types.extend(fuel)


In [272]:
all_vehicles = pd.concat([pd.DataFrame(all_cars),
                          pd.DataFrame(miles).rename(columns={0: 'mileage'}),
                          pd.DataFrame(ages).rename(columns={0: 'reg_year'}),
                          pd.DataFrame(gears).rename(columns={0: 'transmission_type'}),
                          pd.DataFrame(fuel_types).rename(columns={0: 'fuel_type'}),
                          pd.DataFrame(prices).rename(columns={0:'price'})], axis=1)
all_vehicles

,Make,Model,engine_size,trim,mileage,reg_year,transmission_type,fuel_type,price
0,Volkswagen,Golf,1.0,TSI Life 5dr,3962,2022,Manual,Petrol,"20,495"
1,Volkswagen,Tiguan,1.5,TSi EVO 150 Match 5dr DSG,33997,2019,Semiauto,Petrol,"19,795"
2,Kia,Rio,1.25,ISG 2 5dr,76576,2015,Manual,Petrol,"6,395"
3,Volkswagen,Polo,1.2,TSI Match Edition 5dr,40596,2017,Manual,Petrol,"10,795"
4,Hyundai,ix20,1.6,SE 5dr Auto,20712,2016,Automatic,Petrol,"11,595"
...,...,...,...,...,...,...,...,...,...
24095,Volkswagen,T-Roc,1.5,TSI EVO Black Edition 5dr DSG,14599,2021,Semiauto,Petrol,"22,300"
24096,Hyundai,Kona,150kW,Premium 64kWh 5dr Auto,15056,2022,Automatic,Electric,"22,350"
24097,Nissan,X-Trail,1.3,DiG-T Acenta Premium 5dr DCT,13211,2020,Automatic,Petrol,"22,350"
24098,Kia,Sportage,1.6T,GDi ISG GT-Line 5dr DCT Auto [AWD],9599,2021,Automatic,Petrol,"22,400"


In [273]:
all_vehicles.to_csv('../Data/buyacar_data.csv', index=False)